Built an ML model based on** Bidirectional LSTM**(i.e it runs from both forward and backward) which can classify news categories based on their headlines and short description with an accuracy of 96.3%

In [0]:
#downloading the provided data.
!wget --header 'Host: uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) Gecko/20100101 Firefox/63.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://www.dropbox.com/' --header 'Upgrade-Insecure-Requests: 1' 'https://uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com/cd/0/get/AWrcJ4c1Aqo053er_z9yvy5felaDPPLAM_zabPxPuA9Ryhl6v6yyxeCUZmDkyzevMYqScVZQd8o_vF84crp-B5TBAMX7hM98p5_8PZTjWSiZKYZLwuoY3rBR-WLmqJz24ROAhy8A4b2_QKh3AmZs6pjhkbzarGkhHasGV1vBsKd624HZBHzp0F8c68h7nLSyyoE/file?_download_id=318445102638265067571401335412091071113788304135886504837256155&_notify_domain=www.dropbox.com&dl=1' --output-document 'News_Category_Dataset.json.zip'

--2018-12-01 08:11:22--  https://uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com/cd/0/get/AWrcJ4c1Aqo053er_z9yvy5felaDPPLAM_zabPxPuA9Ryhl6v6yyxeCUZmDkyzevMYqScVZQd8o_vF84crp-B5TBAMX7hM98p5_8PZTjWSiZKYZLwuoY3rBR-WLmqJz24ROAhy8A4b2_QKh3AmZs6pjhkbzarGkhHasGV1vBsKd624HZBHzp0F8c68h7nLSyyoE/file?_download_id=318445102638265067571401335412091071113788304135886504837256155&_notify_domain=www.dropbox.com&dl=1
Resolving uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com (uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com)... 162.125.8.6, 2620:100:601b:6::a27d:806
Connecting to uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com (uc7cdc05a7ae86d54cfe46a9baeb.dl.dropboxusercontent.com)|162.125.8.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15551444 (15M) [application/binary]
Saving to: ‘News_Category_Dataset.json.zip’

News_Category_Datas 100%[===================>]  14.83M  53.2MB/s    in 0.3s    

2018-12-01 08:11:23 (53.2 MB/s) - ‘News_Category_Da

In [0]:
!ls

News_Category_Dataset.json  News_Category_Dataset.json.zip  sample_data


In [0]:
#extracting the zip file
!unzip News_Category_Dataset.json.zip

Archive:  News_Category_Dataset.json.zip
  inflating: News_Category_Dataset.json  


In [0]:
#importing basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
df = pd.read_json('News_Category_Dataset.json', lines = True)

In [0]:
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


Combining the headline and short_description as a same text for training.

In [0]:
df['text'] = df['headline'] + ' ' + df['short_description']

Dropping columns which are not required for training till now

In [0]:
df.drop(['authors','date','link','headline','short_description'], axis = 1, inplace = True)

In [0]:
df.head()

,category,text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


In [0]:
df['category'].value_counts()

POLITICS          32739
ENTERTAINMENT     14257
HEALTHY LIVING     6694
QUEER VOICES       4995
BUSINESS           4254
SPORTS             4167
COMEDY             3971
PARENTS            3955
BLACK VOICES       3858
THE WORLDPOST      3664
WOMEN              3490
CRIME              2893
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
IMPACT             2602
WORLDPOST          2579
RELIGION           2556
STYLE              2254
WORLD NEWS         2177
TRAVEL             2145
TASTE              2096
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
SCIENCE            1381
ARTS & CULTURE     1339
TECH               1231
COLLEGE            1144
LATINO VOICES      1129
EDUCATION          1004
Name: category, dtype: int64

THE WORLDPOST and WORLDPOST are the same categories with different names.
Replacing THE WORLDPOST with WORLDPOST

In [0]:
df['category'].replace('THE WORLDPOST','WORLDPOST', inplace = True)

In [0]:
df['category'].value_counts()

POLITICS          32739
ENTERTAINMENT     14257
HEALTHY LIVING     6694
WORLDPOST          6243
QUEER VOICES       4995
BUSINESS           4254
SPORTS             4167
COMEDY             3971
PARENTS            3955
BLACK VOICES       3858
WOMEN              3490
CRIME              2893
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
IMPACT             2602
RELIGION           2556
STYLE              2254
WORLD NEWS         2177
TRAVEL             2145
TASTE              2096
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
SCIENCE            1381
ARTS & CULTURE     1339
TECH               1231
COLLEGE            1144
LATINO VOICES      1129
EDUCATION          1004
Name: category, dtype: int64

Converting label categories into one-hot-encoding by creating dummy variables

In [0]:
Y_oh_train = pd.get_dummies(df.category)

In [0]:
cols = Y_oh_train.columns

In [0]:

import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
!wget --header 'Host: storage.googleapis.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:63.0) Gecko/20100101 Firefox/63.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://www.kaggle.com/' --header 'Upgrade-Insecure-Requests: 1' 'https://storage.googleapis.com/kaggle-datasets/8542/11957/glove.6B.50d.txt.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1543490729&Signature=dTBoFUCG1PLmaozIiMqDoIXWXl7c9J91LgeuL8MHwUD9%2FU88aFJQsKFHl0ippSi7LyGv81j5%2BQx6apAEDRw%2BW3aLgqtNCZ%2FnIKn5IDT%2Fg6af0rbtWpkS8aJTO8mlY7JKOCXXdkIwOnEhq6OBh3oTGyR8plP3jwcYgM4DU89o90VV8Fh%2FiOcZ0Zi8ijoV3oDMhIeBCQ2eAN2ANRwPz5KPsjkRF2xFI8hYllknfIaAnBxAAAwopLoieX8qvcPeUNGUB6PGuOt0HTCiUUOkyBsMa%2FooWY37i%2B6Arv37XprCyhQybi2Qbgp0JIg0UnGu6DYgi9kWRnOh0wWprS%2FgHj5izg%3D%3D' --output-document 'glove.6B.50d.txt.zip'

--2018-11-26 11:25:53--  https://storage.googleapis.com/kaggle-datasets/8542/11957/glove.6B.50d.txt.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1543490729&Signature=dTBoFUCG1PLmaozIiMqDoIXWXl7c9J91LgeuL8MHwUD9%2FU88aFJQsKFHl0ippSi7LyGv81j5%2BQx6apAEDRw%2BW3aLgqtNCZ%2FnIKn5IDT%2Fg6af0rbtWpkS8aJTO8mlY7JKOCXXdkIwOnEhq6OBh3oTGyR8plP3jwcYgM4DU89o90VV8Fh%2FiOcZ0Zi8ijoV3oDMhIeBCQ2eAN2ANRwPz5KPsjkRF2xFI8hYllknfIaAnBxAAAwopLoieX8qvcPeUNGUB6PGuOt0HTCiUUOkyBsMa%2FooWY37i%2B6Arv37XprCyhQybi2Qbgp0JIg0UnGu6DYgi9kWRnOh0wWprS%2FgHj5izg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70948758 (68M) [application/zip]
Saving to: ‘glove.6B.50d.txt.zip’

glove.6B.50d.txt.zi 100%[===================>]  67.66M  58.0MB/s    in 1.2s    

2018-11-26 11:25:55 (58.0 MB/s) - ‘glo

In [0]:
!unzip glove.6B.50d.txt.zip

Archive:  glove.6B.50d.txt.zip
  inflating: glove.6B.50d.txt        


In [0]:
X_train = df["text"].fillna("_na_").values  #replace missing values
y = Y_oh_train[cols].values

In [0]:
!ls

glove.6B.50d.txt      News_Category_Dataset.json      sample_data
glove.6B.50d.txt.zip  News_Category_Dataset.json.zip


In [0]:
maxlen = len(max(X_train, key=len).split()) # max number of words
maxlen 

245

In [0]:
(max(X_train, key=len))

'Sunday Roundup This week the nation watched as the #NeverTrump movement folded faster than one of the presumptive nominee\'s beachfront developments. As many tried to explain away Trump\'s reckless, racist extremism, a few put principle over party. The wife of former Republican Senator Bob Bennett, who died on May 4, revealed that her husband spent his dying hours reaching out to Muslims. "He would go to people with the hijab [on] and tell them he was glad they were in America," she told the Daily Beast. "He wanted to apologize on behalf of the Republican Party." In the U.K., Prime Minister David Cameron called Trump\'s proposal to ban Muslims from entering the U.S., "divisive, stupid and wrong." Trump\'s reply was that he didn\'t think he and Cameron would "have a very good relationship." The press is also doing its part to whitewash extremism. The New York Times called Trump\'s racism "a reductive approach to ethnicity," and said Trump\'s attitude toward women is "complex" and "defi

In [0]:
embed_size = 50 # length of each vector
max_features = 2000 # num rows in embedding vector


Preprocessing sentences using keras.


In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)


In [0]:
EMBEDDING_FILE = 'glove.6B.50d.txt'

Creating Word to Vec dictionary

In [0]:
def get_coefs(word,*arr):
  return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

In [0]:
word_index = tokenizer.word_index
len(word_index)

86627

In [0]:
nb_words = min(max_features, len(word_index))

In [0]:
word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'and': 5,
 'in': 6,
 'is': 7,
 'for': 8,
 'on': 9,
 'that': 10,
 'with': 11,
 'it': 12,
 'you': 13,
 'are': 14,
 'i': 15,
 'this': 16,
 'be': 17,
 'as': 18,
 'at': 19,
 'trump': 20,
 'from': 21,
 'have': 22,
 'not': 23,
 'we': 24,
 'about': 25,
 'was': 26,
 'new': 27,
 'his': 28,
 'your': 29,
 'but': 30,
 'has': 31,
 'an': 32,
 'what': 33,
 'will': 34,
 'by': 35,
 'how': 36,
 'my': 37,
 'he': 38,
 'one': 39,
 'all': 40,
 'more': 41,
 'who': 42,
 'can': 43,
 'their': 44,
 'up': 45,
 'after': 46,
 'people': 47,
 'out': 48,
 'our': 49,
 'her': 50,
 'just': 51,
 'they': 52,
 'when': 53,
 'or': 54,
 'donald': 55,
 "it's": 56,
 'like': 57,
 'if': 58,
 'time': 59,
 'over': 60,
 'so': 61,
 'do': 62,
 'no': 63,
 'why': 64,
 'says': 65,
 'than': 66,
 'said': 67,
 'year': 68,
 'first': 69,
 'been': 70,
 'u': 71,
 'president': 72,
 'day': 73,
 's': 74,
 'get': 75,
 'world': 76,
 'women': 77,
 'life': 78,
 'some': 79,
 'most': 80,
 'now': 81,
 'make': 82,
 '

In [0]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

Creating a bidirectional LSTM 

In [0]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.2))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(30, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit(X_t, y, batch_size=32, epochs=1, validation_split=0.1);

KeyboardInterrupt: ignored